In [ ]:
from torchvision import datasets, transforms
import torchvision
import torch
import torch.optim as optim
from skorch import NeuralNetClassifier

from itertools import islice
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# add root project to path for imports, this issue is just for notebook environment
import os, sys
from pathlib import Path

def get_project_path(project_name: str) -> Path:
    path = Path(os.path.abspath(''))
    previous_path = None  # Use to prevent infinite loop in case of error
    while path.name != project_name:
        path = path.parent
        if path == previous_path:
            raise ValueError(f"Project '{project_name}' not found in path hierarchy.")
        previous_path = path
    return path

try:
    project_name = "AIProjects"
    project_path = get_project_path(project_name)
    sys.path.append(str(project_path))
except ValueError as e:
    print(f"Project '{project_name}' not found in path hierarchy.")
    try:
        project_name = "machine-learning-portfolio"
        project_path = get_project_path(project_name)
        sys.path.append(str(project_path))
    except ValueError:
        print(f"Project '{project_name}' not found in path hierarchy.")


In [ ]:
# Transformations de base (obligatoires)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2470, 0.2435, 0.2616)
    )
])

train_dataset = datasets.CIFAR10(
    root="../data",
    train=True,
    transform=transform,
    download=True
)

test_dataset = datasets.CIFAR10(
    root="../data",
    train=False,
    transform=transform,
    download=True
)

# Visualize some images

In [ ]:
X_example, y_example = zip(*islice(iter(train_dataset), 7))

In [ ]:
X_example[0].min(), X_example[0].max()

In [ ]:
y_example

In [ ]:
def plot_example(X, y, n=5):
    """Plot the images in X and their labels in rows of `n` elements."""
    fig = plt.figure(figsize=(10, 10))
    rows = len(X) // n + 1
    for i, (img, y) in enumerate(zip(X, y)):
        ax = fig.add_subplot(rows, n, i + 1)
        ax.imshow(img.permute(1, 2, 0).numpy() / 2 + 0.5)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(train_dataset.classes[y])
    plt.tight_layout()
    return fig

In [ ]:
plot_example(torch.stack(X_example), y_example, n=7);

# Distribution of classes in the training set

In [ ]:
import seaborn as sns

label_counts = dict()
for _, label in train_dataset:
    label_name = train_dataset.classes[label]
    if label_name in label_counts:
        label_counts[label_name] += 1
    else:
        label_counts[label_name] = 1

sns.barplot(x=list(label_counts.keys()), y=list(label_counts.values()))
plt.xticks(rotation=45)
plt.title("Distribution of classes in the training set")
plt.show()

# Training a baseline model

In [ ]:
y_train = np.array([y for x, y in iter(train_dataset)])
y_test = np.array([y for x, y in iter(test_dataset)])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from computer_vision.src.baseline import BaselineModel

In [ ]:
from torch import nn
import torch.nn.functional as F

In [ ]:
torch.manual_seed(0)

baseline = NeuralNetClassifier(
    BaselineModel,
    max_epochs=10,
    iterator_train__num_workers=2,
    iterator_valid__num_workers=2,
    lr=0.1,
    device=device,
    module__input_dim=32*32*3,
)

In [ ]:
baseline.fit(train_dataset, y=y_train);

## Plot training loss

In [ ]:
loss_history = baseline.history[:, 'train_loss']
sns.lineplot(x=range(1, len(loss_history) + 1), y=loss_history)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss over Epochs")
plt.show()

## Evaluating the model

In [ ]:
print("Test set Accuracy: {:.2f}%".format(
    100 * baseline.score(test_dataset, y_test)
))

# Use BaseCNN with custom parameters

In [ ]:
from utils.BaseCNN import BaseCNN
from sklearn.model_selection import GridSearchCV
from skorch.helper import SliceDataset


cnn = NeuralNetClassifier(
    BaseCNN,
    max_epochs=10,
    lr=0.001,
    optimizer=optim.Adam,
    device=device,
    module__img_height=32,
    module__img_width=32,
    module__nb_conv_layers=2,
    module__nb_layers=2,
    module__net_width=256,
    module__dropout_rates=[0.25, 0.5],
)


In [ ]:
cnn.fit(train_dataset, y_train)

In [ ]:
loss_history = cnn.history[:, 'train_loss']

sns.lineplot(x=range(1, len(loss_history) + 1), y=loss_history)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss over Epochs")
plt.show()

In [ ]:
print("Test set Accuracy: {:.2f}%".format(
    100 * cnn.score(test_dataset, y_test)
))

# Hyperparameter Tuning with Grid Search

In [ ]:
net = NeuralNetClassifier(
    BaseCNN,
    max_epochs=10,
    lr=0.1,

)

In [ ]:
net.set_params(train_split=False, verbose=0)
params = {
    'lr': [0.01, 0.02],
    'max_epochs': [10, 20],
    'optimizer': ['Adam', 'SGD'],
    #'module__img_height':32,
    #'module__img_width':32,
    'module__nb_conv_layers':[2, 3],
    'module__nb_layers':[2, 3],
    'module__net_width':[128, 256, 512],
    'module__dropout_rates':[[0],[0.25, 0.5]],
}

In [ ]:
grid = GridSearchCV(net, params, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)

In [ ]:
train_dataset_sliceable = SliceDataset(train_dataset)

In [ ]:
grid.fit(train_dataset_sliceable, y_train)

In [ ]:
print("Best parameters found: ", grid.best_params_)
print("Best cross-validation accuracy: ", grid.best_score_)

In [ ]:
print("Test set accuracy: ", grid.score(SliceDataset(test_dataset), y_test))